In [1]:
!pip install transformers -q

In [2]:
import numpy as np
import pandas as pd
import transformers
import torch
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch import nn
import warnings
warnings.filterwarnings("ignore")

In [3]:
from transformers import BertModel, BertTokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
bert = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

#### Loading the dataset and pre-processing

In [6]:
data = pd.read_csv('/kaggle/input/forc-data/train.csv')

In [7]:
data

,abstract,author,doi,url,publication month,publication year,title,publisher,label,data_index
0,the production of b jets in association with a...,CMS Collaboration,10.1007/JHEP06(2012)126,NaN,NaN,NaN,Measurement of the Z/gamma* + b-jet cross sect...,JHEP 06 (2012) 126,Physics,44436
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",NaN,NaN,NaN,NaN,Modelling systemic price cojumps with Hawkes f...,NaN,Quantitative Finance,46308
2,large information sizes in samples and feature...,"David Banh, Alan Huang",NaN,NaN,NaN,NaN,Encoding large information structures in linea...,NaN,Machine Learning,52468
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",NaN,NaN,NaN,NaN,Hyperbolic polygonal billiards close to 1-dime...,NaN,Dynamics/Dynamical Systems,12449
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",10.1016/j.jag.2012.08.003,https://doi.org/10.1016/j.jag.2012.08.003,4,2013.0,analysis of aster data for mapping bauxite ric...,International Journal of Applied Earth Observa...,Sociology,43248
...,...,...,...,...,...,...,...,...,...,...
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",10.1002/pmj.21366,NaN,10,2013.0,recruiting project managers: a comparative ana...,NaN,Sociology,52038
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),NaN,NaN,NaN,NaN,Optimal stopping for L\'evy processes and affi...,NaN,Statistics and Probability,20925
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",10.1103/PhysRevD.78.075006,NaN,NaN,NaN,Against Tachyophobia,"Phys.Rev.D78:075006,2008",Physics,16875
41538,NaN,"['Markus Mühling ', 'Ralph Ewerth', 'Joanna Ba...",10.1007/978-3-030-30760-8_9,NaN,9,2019.0,investigating correlations of inter-coder agre...,NaN,Computer Sciences,46341


In [8]:
data.columns

Index(['abstract', 'author', 'doi', 'url', 'publication month',
       'publication year', 'title', 'publisher', 'label', 'data_index'],
      dtype='object')

In [9]:
data.isnull().sum()

abstract              1120
author                 189
doi                  16977
url                  39593
publication month    34943
publication year     34936
title                    0
publisher            23942
label                    0
data_index               0
dtype: int64

In [10]:
data.drop(['doi','url','publication month', 'publication year','publisher', 'data_index'], axis =1 , inplace = True)
data

,abstract,author,title,label
0,the production of b jets in association with a...,CMS Collaboration,Measurement of the Z/gamma* + b-jet cross sect...,Physics
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",Modelling systemic price cojumps with Hawkes f...,Quantitative Finance
2,large information sizes in samples and feature...,"David Banh, Alan Huang",Encoding large information structures in linea...,Machine Learning
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",Hyperbolic polygonal billiards close to 1-dime...,Dynamics/Dynamical Systems
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",analysis of aster data for mapping bauxite ric...,Sociology
...,...,...,...,...
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",recruiting project managers: a comparative ana...,Sociology
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),Optimal stopping for L\'evy processes and affi...,Statistics and Probability
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",Against Tachyophobia,Physics
41538,NaN,"['Markus Mühling ', 'Ralph Ewerth', 'Joanna Ba...",investigating correlations of inter-coder agre...,Computer Sciences


In [11]:
data.isnull().sum()

abstract    1120
author       189
title          0
label          0
dtype: int64

In [12]:
data = data.dropna()
data.isnull().sum()

abstract    0
author      0
title       0
label       0
dtype: int64

In [13]:
data.shape

(40332, 4)

In [14]:
data

,abstract,author,title,label
0,the production of b jets in association with a...,CMS Collaboration,Measurement of the Z/gamma* + b-jet cross sect...,Physics
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",Modelling systemic price cojumps with Hawkes f...,Quantitative Finance
2,large information sizes in samples and feature...,"David Banh, Alan Huang",Encoding large information structures in linea...,Machine Learning
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",Hyperbolic polygonal billiards close to 1-dime...,Dynamics/Dynamical Systems
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",analysis of aster data for mapping bauxite ric...,Sociology
...,...,...,...,...
41534,programs offered by academic institutions in h...,"['Alex Ferworn', 'Muthana Zouri']",an ontology-based approach for curriculum mapp...,Computer Engineering
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",recruiting project managers: a comparative ana...,Sociology
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),Optimal stopping for L\'evy processes and affi...,Statistics and Probability
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",Against Tachyophobia,Physics


In [15]:
def NLP_cleaning(text):
    text_corpus = []
    for sent in tqdm(text, desc='Cleaning'):
        sent = re.sub('<[^>]*>', '', sent)
        sent = re.sub('[^a-zA-z0-9]', ' ', sent)
        sent = sent.lower()
        text_corpus.append(sent)

    return text_corpus

In [16]:
text = data.abstract.values.tolist()

In [17]:
text[0]

'the production of b jets in association with a z/gamma* boson is studied using proton-proton collisions delivered by the lhc at a centre-of-mass energy of 7 tev and recorded by the cms detector. the inclusive cross section for z/gamma* + b-jet production is measured in a sample corresponding to an integrated luminosity of 2.2 inverse femtobarns. the z/gamma* + b-jet cross section with z/gamma* to ll (where ll = ee or mu mu) for events with the invariant mass 60 < m(ll) < 120 gev, at least one b jet at the hadron level with pt > 25 gev and abs(eta) < 2.1, and a separation between the leptons and the jets of delta r > 0.5 is found to be 5.84 +/- 0.08 (stat.) +/- 0.72 (syst.) +(0.25)/-(0.55) (theory) pb. the kinematic properties of the events are also studied and found to be in agreement with the predictions made by the madgraph event generator with the parton shower and the hadronisation performed by pythia.'

In [18]:
text_corpus = NLP_cleaning(text)

Cleaning: 100%|██████████| 40332/40332 [00:03<00:00, 12655.43it/s]


In [19]:
text_corpus[0]

'the production of b jets in association with a z gamma  boson is studied using proton proton collisions delivered by the lhc at a centre of mass energy of 7 tev and recorded by the cms detector  the inclusive cross section for z gamma    b jet production is measured in a sample corresponding to an integrated luminosity of 2 2 inverse femtobarns  the z gamma    b jet cross section with z gamma  to ll  where ll   ee or mu mu  for events with the invariant mass 60  25 gev and abs eta   0 5 is found to be 5 84     0 08  stat       0 72  syst     0 25    0 55   theory  pb  the kinematic properties of the events are also studied and found to be in agreement with the predictions made by the madgraph event generator with the parton shower and the hadronisation performed by pythia '

In [20]:
data['abstract'] = text_corpus

In [21]:
data['title'] = NLP_cleaning(data.title.values.tolist())
data['author'] = NLP_cleaning(data.author.values.tolist())

Cleaning: 100%|██████████| 40332/40332 [00:00<00:00, 83516.25it/s]


In [22]:
data

,abstract,author,title,label
0,the production of b jets in association with a...,cms collaboration,measurement of the z gamma b jet cross sect...,Physics
1,instabilities in the price dynamics of a large...,giacomo bormetti lucio maria calcagnile mich...,modelling systemic price cojumps with hawkes f...,Quantitative Finance
2,large information sizes in samples and feature...,david banh alan huang,encoding large information structures in linea...,Machine Learning
3,we consider polygonal billiards with collision...,gianluigi del magno jo\ ao lopes dias pedro ...,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems
4,bauxite deposits of jharkhand in india are res...,[ e n dhanamjaya rao a t jeyaseelan k ...,analysis of aster data for mapping bauxite ric...,Sociology
...,...,...,...,...
41534,programs offered by academic institutions in h...,[ alex ferworn muthana zouri ],an ontology based approach for curriculum mapp...,Computer Engineering
41535,this research addresses the competencies organ...,[ sabik khan marcus ho kamrul ahsan ],recruiting project managers a comparative ana...,Sociology
41536,this paper studies an optimal stopping problem...,diana dorobantu lsproba,optimal stopping for l\ evy processes and affi...,Statistics and Probability
41537,we examine the possible extension of the param...,john ellis joel giedt oleg lebedev keith ol...,against tachyophobia,Physics


In [23]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['label_number'] = label_encoder.fit_transform(data['label'])
data

,abstract,author,title,label,label_number
0,the production of b jets in association with a...,cms collaboration,measurement of the z gamma b jet cross sect...,Physics,95
1,instabilities in the price dynamics of a large...,giacomo bormetti lucio maria calcagnile mich...,modelling systemic price cojumps with hawkes f...,Quantitative Finance,105
2,large information sizes in samples and feature...,david banh alan huang,encoding large information structures in linea...,Machine Learning,67
3,we consider polygonal billiards with collision...,gianluigi del magno jo\ ao lopes dias pedro ...,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems,41
4,bauxite deposits of jharkhand in india are res...,[ e n dhanamjaya rao a t jeyaseelan k ...,analysis of aster data for mapping bauxite ric...,Sociology,114
...,...,...,...,...,...
41534,programs offered by academic institutions in h...,[ alex ferworn muthana zouri ],an ontology based approach for curriculum mapp...,Computer Engineering,25
41535,this research addresses the competencies organ...,[ sabik khan marcus ho kamrul ahsan ],recruiting project managers a comparative ana...,Sociology,114
41536,this paper studies an optimal stopping problem...,diana dorobantu lsproba,optimal stopping for l\ evy processes and affi...,Statistics and Probability,118
41537,we examine the possible extension of the param...,john ellis joel giedt oleg lebedev keith ol...,against tachyophobia,Physics,95


In [24]:
val_df = pd.read_csv('/kaggle/input/forc-data/val.csv')
val_df

,abstract,author,doi,url,publication month,publication year,title,publisher,label,data_index
0,we use high-resolution zoom-in cosmological si...,"Hidenobu Yajima (1), Isaac Shlosman (2, 3), Em...",10.1093/mnras/stv974,NaN,NaN,NaN,Observational Properties of Simulated Galaxies...,NaN,External Galaxies,22292
1,NaN,"['Yongzhuo Huang', 'Shengwu Huang', 'Yongxin Y...",10.1007/s13233-013-1029-2,NaN,10.0,2012.0,cell-penetrating peptide-modified plga nanopar...,Macromolecular Research,Medicinal Chemistry and Pharmaceutics,4465
2,it is shown that the formulation of the su(3) ...,Hans-Peter Pavel,NaN,NaN,NaN,NaN,SU(3) Yang-Mills Hamiltonian in the flux-tube ...,NaN,Quantum Physics,16166
3,"structural, magnetization, phonon behavior, an...","Ilyas Noor Bhatti, Imtiaz Noor Bhatti, Rabindr...",NaN,NaN,NaN,NaN,Suppression of magnetic phase transition at hi...,NaN,"Atomic, Molecular and Optical Physics",19452
4,the result of padrol asserts that for every $d...,Isabella Novik and Hailun Zheng,NaN,NaN,NaN,NaN,Many neighborly spheres,NaN,Discrete Mathematics and Combinatorics,43731
...,...,...,...,...,...,...,...,...,...,...
8896,joint energy distribution of the bottom quark ...,Ivan V. Truten and Alexander Yu. Korchin,10.1088/1361-6471/ac4e60,NaN,NaN,NaN,Energy correlation of bottom quarks from decay...,J. Phys. G: Nucl. Part. Phys. 49 045003 (2022),Physics,49058
8897,"the elastic scattering, stark transitions and ...",V.P. Popov and V.N. Pomerantsev,NaN,NaN,NaN,NaN,Scattering processes in antiprotonic hydrogen ...,NaN,Nonlinear Sciences,26605
8898,We identify privacy risks associated with rele...,"['Philipp Weis', 'Don Towsley', 'David Jensen'...",10.14778/1453856.1453873,NaN,8.0,2008.0,resisting structural re-identification in anon...,NaN,Earth Sciences,44741
8899,Predictive models of immune protection from CO...,"['Miles P. Davenport', 'James A. Triccas', 'St...",10.1038/s41591-021-01377-8,http://dx.doi.org/10.1038/s41591-021-01377-8,5.0,2021.0,neutralizing antibody levels are highly predic...,Nature Medicine,Microbiology,19564


In [25]:
val_df.drop(['doi','url','publication month', 'publication year','publisher', 'data_index'], axis =1 , inplace = True)
val_df = val_df.dropna()
val_df['title'] = NLP_cleaning(val_df.title.values.tolist())
val_df['author'] = NLP_cleaning(val_df.author.values.tolist())
val_df['abstract'] = NLP_cleaning(val_df.title.values.tolist())
val_df

Cleaning: 100%|██████████| 8648/8648 [00:00<00:00, 106327.47it/s]


,abstract,author,title,label
0,observational properties of simulated galaxies...,hidenobu yajima 1 isaac shlosman 2 3 em...,observational properties of simulated galaxies...,External Galaxies
2,su 3 yang mills hamiltonian in the flux tube ...,hans peter pavel,su 3 yang mills hamiltonian in the flux tube ...,Quantum Physics
3,suppression of magnetic phase transition at hi...,ilyas noor bhatti imtiaz noor bhatti rabindr...,suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics"
4,many neighborly spheres,isabella novik and hailun zheng,many neighborly spheres,Discrete Mathematics and Combinatorics
5,physical properties of reb2 under high pressur...,ming min zhong xiao yu kuang zhen hua wang ...,physical properties of reb2 under high pressur...,Materials Science and Engineering
...,...,...,...,...
8895,spectral properties of ngc 4151 and the estima...,prantik nandi sandip k chakrabarti santanu ...,spectral properties of ngc 4151 and the estima...,Physical Processes
8896,energy correlation of bottom quarks from decay...,ivan v truten and alexander yu korchin,energy correlation of bottom quarks from decay...,Physics
8897,scattering processes in antiprotonic hydrogen ...,v p popov and v n pomerantsev,scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences
8898,resisting structural re identification in anon...,[ philipp weis don towsley david jensen ...,resisting structural re identification in anon...,Earth Sciences


In [26]:
val_df['label_number'] = label_encoder.transform(val_df['label'])
val_df

,abstract,author,title,label,label_number
0,observational properties of simulated galaxies...,hidenobu yajima 1 isaac shlosman 2 3 em...,observational properties of simulated galaxies...,External Galaxies,51
2,su 3 yang mills hamiltonian in the flux tube ...,hans peter pavel,su 3 yang mills hamiltonian in the flux tube ...,Quantum Physics,107
3,suppression of magnetic phase transition at hi...,ilyas noor bhatti imtiaz noor bhatti rabindr...,suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics",10
4,many neighborly spheres,isabella novik and hailun zheng,many neighborly spheres,Discrete Mathematics and Combinatorics,40
5,physical properties of reb2 under high pressur...,ming min zhong xiao yu kuang zhen hua wang ...,physical properties of reb2 under high pressur...,Materials Science and Engineering,68
...,...,...,...,...,...
8895,spectral properties of ngc 4151 and the estima...,prantik nandi sandip k chakrabarti santanu ...,spectral properties of ngc 4151 and the estima...,Physical Processes,93
8896,energy correlation of bottom quarks from decay...,ivan v truten and alexander yu korchin,energy correlation of bottom quarks from decay...,Physics,95
8897,scattering processes in antiprotonic hydrogen ...,v p popov and v n pomerantsev,scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences,84
8898,resisting structural re identification in anon...,[ philipp weis don towsley david jensen ...,resisting structural re identification in anon...,Earth Sciences,42


In [27]:
train_text, val_text, train_labels, val_labels = train_test_split(data.drop(['label','label_number'], axis = 1), data['label_number'],
                                                                    random_state=2018,
                                                                    test_size=0.2,
                                                                    stratify=data['label_number'])


test_text = val_df.drop(['label','label_number'], axis = 1)
test_labels = val_df['label_number']

In [28]:
test_text

,abstract,author,title
0,observational properties of simulated galaxies...,hidenobu yajima 1 isaac shlosman 2 3 em...,observational properties of simulated galaxies...
2,su 3 yang mills hamiltonian in the flux tube ...,hans peter pavel,su 3 yang mills hamiltonian in the flux tube ...
3,suppression of magnetic phase transition at hi...,ilyas noor bhatti imtiaz noor bhatti rabindr...,suppression of magnetic phase transition at hi...
4,many neighborly spheres,isabella novik and hailun zheng,many neighborly spheres
5,physical properties of reb2 under high pressur...,ming min zhong xiao yu kuang zhen hua wang ...,physical properties of reb2 under high pressur...
...,...,...,...
8895,spectral properties of ngc 4151 and the estima...,prantik nandi sandip k chakrabarti santanu ...,spectral properties of ngc 4151 and the estima...
8896,energy correlation of bottom quarks from decay...,ivan v truten and alexander yu korchin,energy correlation of bottom quarks from decay...
8897,scattering processes in antiprotonic hydrogen ...,v p popov and v n pomerantsev,scattering processes in antiprotonic hydrogen ...
8898,resisting structural re identification in anon...,[ philipp weis don towsley david jensen ...,resisting structural re identification in anon...


In [29]:
train_text

,abstract,author,title
32105,phd thesis a compilation of the papers lowe...,brendon stanton,on vertex identifying codes for infinite lattices
30882,aim to help people living with hiv aids gain ...,[ do aner g b g kengin s karabilgin ],g kengin d the effect of psychodrama on peop...
24200,we determine the dependence of the initial hel...,aldo serenelli sarbani basu,determining the initial helium abundance of th...
28595,in this paper we propose a novel representati...,radu tudor ionescu and andrei m butnaru,vector of locally aggregated word embeddings ...
2594,classical approaches like process algebras or ...,christian attiogb\ e lina,dynamic composition of evolving process types
...,...,...,...
32986,generation of primordial magnetic fields durin...,kumar atmjeet isha pahwa t r seshadri and ...,cosmological magnetogenesis from extra dimensi...
28799,a general formulation of stochastic thermodyna...,massimiliano esposito,stochastic thermodynamics under coarse graining
18559,the least significant bit lsb based approach...,[ jiwu huang fangjun huang weiqi luo ],edge adaptive image steganography based on lsb...
27438,this paper is devoted to geodesic completeness...,ahmed elshafei ana cristina ferreira and hele...,geodesic completeness of pseudo and holomorphi...


In [30]:
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
sent_id = tokenizer.batch_encode_plus(text, padding=True)
print(sent_id)

{'input_ids': [[101, 2023, 2003, 1037, 14324, 2944, 14924, 4818, 102, 0], [101, 2057, 2097, 2986, 1011, 8694, 1037, 14324, 2944, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [31]:
tokens_train = tokenizer.batch_encode_plus(
    train_text['abstract'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

tokens_val = tokenizer.batch_encode_plus(
    val_text['abstract'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

tokens_test = tokenizer.batch_encode_plus(
    test_text['abstract'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

In [32]:
len(pd.unique(data['label_number']))

123

In [33]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [34]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [35]:
for param in bert.parameters():
    param.requires_grad = False

In [36]:
import torch
import torch.nn as nn
from transformers import BertModel

class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 123)  # Adjust the number of output neurons for 123 classes
        self.log_softmax = nn.LogSoftmax(dim=1)  # Use dim=1 for LogSoftmax in classification

    def forward(self, sent_id, mask):
        sent_id = torch.tensor(sent_id)

        outputs = self.bert(input_ids=sent_id, attention_mask=mask)
        last_hidden_state_cls = outputs[0][:, 0, :]

        x = self.fc1(last_hidden_state_cls)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.log_softmax(x)

        return x

In [37]:
model = BERT_Arch(bert)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [38]:
device

'cuda'

In [39]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = 1e-5)

In [40]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(train_labels)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_labels)
print("Class Weights:", class_weights)

Class Weights: [6.79577910e-01 5.90804219e-01 5.46493902e-01 5.14347202e+00
 5.34250658e-01 2.42886179e+00 7.14760417e-01 6.24564460e+00
 1.25932344e-01 8.74390244e+00 3.48362647e-01 2.62317073e+01
 3.72080955e-01 2.01782364e+00 5.24634146e+01 8.46184107e+00
 4.37195122e+01 1.68151970e+00 3.27896341e+01 1.38061617e+01
 1.82164634e+00 7.49477352e+00 5.24634146e+00 4.53051940e-01
 5.82926829e+00 2.30102696e+00 3.80169671e+00 7.67008986e-01
 2.80253283e-01 2.36321688e+00 2.49825784e+00 3.74204099e-01
 3.97450111e+00 5.56936461e-01 3.95651694e-01 8.40759850e-01
 1.38061617e+01 1.38792102e+00 6.32089333e-01 6.10039705e+00
 4.15716439e-01 2.70429972e-01 4.23092054e+00 1.59949435e+00
 1.24912892e+01 1.63948171e+01 2.47468937e+00 9.04541632e+00
 1.31158537e+02 1.84730333e+00 4.09870427e+00 4.23092054e-01
 1.18695508e+00 1.19235033e+01 3.51631465e-01 5.82926829e+00
 1.19235033e+01 8.46184107e+00 1.19235033e+01 2.38470067e+00
 8.74390244e+00 1.05348222e+00 3.74738676e+00 9.40204563e-01
 2.790607

In [41]:
weights= torch.tensor(class_weights,dtype=torch.float)
weights = weights.to(device)
cross_entropy  = nn.NLLLoss(weight=weights)

epochs = 10

In [42]:
def train():

    model.train()
    total_loss, total_accuracy = 0, 0
    total_preds=[]

    for step,batch in enumerate(train_dataloader):

        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch
        sent_id = torch.tensor(sent_id)

        model.zero_grad()
        preds = model(sent_id, mask)
        loss = cross_entropy(preds, labels)
        total_loss = total_loss + loss.item()
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        preds=preds.detach().cpu().numpy()
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [43]:
import time
def evaluate():

    print("\nEvaluating...")
    model.eval()
    total_loss, total_accuracy = 0, 0
    total_preds = []

    for step,batch in enumerate(val_dataloader):

        if step % 50 == 0 and not step == 0:
#             elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch
    sent_id = torch.tensor(sent_id)

    with torch.no_grad():

        preds = model(sent_id, mask)
        loss = cross_entropy(preds,labels)
        total_loss = total_loss + loss.item()
        preds = preds.detach().cpu().numpy()
        total_preds.append(preds)

    avg_loss = total_loss / len(val_dataloader)

    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [44]:
best_valid_loss = float('inf')

train_losses=[]
valid_losses=[]

for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, _ = train()
    valid_loss, _ = evaluate()
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of  2,017.
  Batch   100  of  2,017.
  Batch   150  of  2,017.
  Batch   200  of  2,017.
  Batch   250  of  2,017.
  Batch   300  of  2,017.
  Batch   350  of  2,017.
  Batch   400  of  2,017.
  Batch   450  of  2,017.
  Batch   500  of  2,017.
  Batch   550  of  2,017.
  Batch   600  of  2,017.
  Batch   650  of  2,017.
  Batch   700  of  2,017.
  Batch   750  of  2,017.
  Batch   800  of  2,017.
  Batch   850  of  2,017.
  Batch   900  of  2,017.
  Batch   950  of  2,017.
  Batch 1,000  of  2,017.
  Batch 1,050  of  2,017.
  Batch 1,100  of  2,017.
  Batch 1,150  of  2,017.
  Batch 1,200  of  2,017.
  Batch 1,250  of  2,017.
  Batch 1,300  of  2,017.
  Batch 1,350  of  2,017.
  Batch 1,400  of  2,017.
  Batch 1,450  of  2,017.
  Batch 1,500  of  2,017.
  Batch 1,550  of  2,017.
  Batch 1,600  of  2,017.
  Batch 1,650  of  2,017.
  Batch 1,700  of  2,017.
  Batch 1,750  of  2,017.
  Batch 1,800  of  2,017.
  Batch 1,850  of  2,017.
  Batch 1,900  of  2,01

In [45]:
checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

In [46]:
input_text = input('Give an input text : ')
tokens = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

Give an input text :  physics is love


In [47]:
def Predict(text):
    encoded_review = tokenizer.encode_plus(
      text,
      max_length=256,
      add_special_tokens=True,
      return_token_type_ids=False,
      truncation = True,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    extracted_value = prediction.item()
    return extracted_value

In [48]:
y_pred = []
for text in test_text['abstract']:
    val = Predict(text)
    y_pred.append(val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [49]:
len(y_pred)

8648

In [50]:
test_labels

0        51
2       107
3        10
4        40
5        68
       ... 
8895     93
8896     95
8897     84
8898     42
8899     77
Name: label_number, Length: 8648, dtype: int64

In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef, classification_report
import numpy as np

true_labels = test_labels
predicted_labels = y_pred

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.4339


In [52]:
print(classification_report(true_labels, y_pred))

              precision    recall  f1-score   support

           0       0.21      0.64      0.31        99
           1       0.42      0.61      0.50       126
           2       0.25      0.52      0.34       130
           3       0.48      0.83      0.61        12
           4       0.21      0.18      0.19       118
           5       0.15      0.13      0.14        30
           6       0.16      0.33      0.22       104
           7       0.38      0.33      0.35         9
           8       0.45      0.47      0.46       557
           9       0.20      0.18      0.19        11
          10       0.56      0.34      0.42       225
          11       0.25      0.33      0.29         3
          12       0.85      0.38      0.53       189
          13       0.24      0.19      0.21        42
          14       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         9
          16       0.50      0.50      0.50         2
          17       0.26    